In [15]:
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from shapely.strtree import STRtree
import pickle
import osmnx as ox
import folium
file_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph.pickle'

with open(file_path, 'rb') as file:
    G = pickle.load(file)
G.graph['crs'] = 'epsg:4326'
# 장
obstacle_point = Point(	37.377069, 126.634596)


# 장애물 가까이에 있는 엣지 가중치 증가 함수 정의
def update_edge_weights(G, point, increment=5):
    # 가장 가까운 엣지 찾기
    nearest_edge_nodes = ox.distance.nearest_edges(G, X=point.y, Y=point.x)  # OSMnx는 (y, x) 형식으로 좌표를 사용합니다.
    print(nearest_edge_nodes)
    # nearest_edge_nodes는 (u, v, key) 형태의 튜플입니다.
    u, v, key = nearest_edge_nodes

    # 해당 엣지의 데이터 가져오기
    nearest_edge = G[u][v][key]
    print(nearest_edge)
    # 가장 가까운 엣지의 가중치 업데이트
    if nearest_edge:
        # 엣지 데이터에 'weight' 키가 없으면 기본값으로 0을 설정
        #if 'weight' not in nearest_edge:
            #nearest_edge['weight'] = 0
        nearest_edge['weight'] += increment
        print("change")
        print(nearest_edge)
        return (u, v, key), nearest_edge
    
m = folium.Map(location=[37.37463, 126.64481], zoom_start=14)    


modified_edge, edge_data = update_edge_weights(G, obstacle_point)
combined_line_string = edge_data['geometry']


# 합쳐진 LineString을 Folium 지도에 추가 (비어 있지 않은 경우에만 추가)
if combined_line_string:
    folium.GeoJson(combined_line_string, style_function=lambda x: {'color': 'blue', 'weight': 5}).add_to(m)
else:
    print("지도에 추가할 LineString이 없습니다.")
# 시작 지점에 마커 추가
folium.Marker(
    location=[obstacle_point.x,obstacle_point.y],
    icon=folium.Icon(color='red')
).add_to(m)

m

('28000761546', '28000762852', 0)
{'link_id': '28200362607', 'weight': 63.702618998460366, 'geometry': <LINESTRING (126.635 37.377, 126.635 37.377, 126.635 37.377, 126.635 37.377,...>}
change
{'link_id': '28200362607', 'weight': 68.70261899846037, 'geometry': <LINESTRING (126.635 37.377, 126.635 37.377, 126.635 37.377, 126.635 37.377,...>}


In [13]:
import pandas as pd

# Load the DataFrame from the pickle file
pickle_file_path = 'C:/my_code/moduroad/data/obstacle/slope.pkl'  # Replace with your pickle file path
df = pd.read_pickle(pickle_file_path)

# Convert the DataFrame to a CSV file
csv_file_path = 'C:/my_code/moduroad/data/obstacle/slope.csv'  # Replace with your desired CSV file path
df.to_csv(csv_file_path, index=False,encoding='utf-8-sig')  # Set index=False if you don't want to include the index in the CSV
